# To investigate model-stitching analysis on VGG19
Stitch randomly initialised model into VGG19 unbias receiver - based on exp2j which self-stitched models. In this case, use the bias (correlated) dataset to see how that affects the ability to stitch in the random model

In [1]:
from google.colab import files
from google.colab import drive

drive.mount('/content/drive')
!pip install torchbearer


Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitli

In [2]:
# Packages
%matplotlib inline

drive_path = '/content/drive/MyDrive/Colab Notebooks/icml'
import os
os.chdir(drive_path)

import argparse
import gc
import os.path

import pandas as pd
from torch.linalg import LinAlgError

import matplotlib.pyplot as plt
import torchvision
import torch
from torch import optim

from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import datetime

import random
import numpy as np

import sys
import os
# add the path to find colour_mnist
#sys.path.append(os.path.abspath('../ReferenceCode'))
import colour_mnist
from stitch_utils import train_model, StitchedVGG19, get_layer_output_shape
from stitch_utils import generate_activations, SyntheticDataset
import stitch_utils

# add the path to find the rank analysis code
# https://github.com/DHLSmith/jons-tunnel-effect/tree/NeurIPSPaper

#sys.path.append(os.path.abspath('../../jons-tunnel-effect/'))
from utils.modelfitting import evaluate_model, set_seed
from extract_weight_rank import install_hooks, perform_analysis

import torchvision
from torchvision.datasets import MNIST

def logtofile(log_text, verbose=True):
    if verbose:
        print(log_text)
    with open(save_log_as, "a") as f:
        print(log_text, file=f)

In [3]:
# Set Parameters

# fix random seed for reproducibility
seed = 107
torch.manual_seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
random.seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

train_all = False
save_stitch_delta = False
measure_rank = False

results_root = "results_2j_b"

# randinit model
gen_randinit_model = False
randinit_model_to_load = '2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit.weights'

# MIX is 1/3 bgonly, 1/3 mnist only, 1/3 biased data
train_mix_mnist_model = train_all
mix_mnist_model_to_load = '../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_mix_mnist.weights'

# BW is greyscale mnist with no colour added (i.e. original mnist)
train_bw_mnist_model = train_all  # when False, automatically loads a trained model
bw_mnist_model_to_load = '../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_bw_mnist.weights'

# BG_ONLY contains no mnist data, just a coloured background
train_bg_only_colour_mnist_model = train_all # when False, automatically loads a trained model
bg_only_colour_mnist_model_to_load     =  '../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_bg_only_colour_mnist.weights'
#bg_only_std_colour_mnist_model_to_load =  '../exp1_ms_with_random_dataset/results_1g/2024-08-10_22-57-13_SEED104_EPOCHS50_BGN0.1_exp1g_VGG19_bg_only_colour_mnist.weights'

# BG_UNBIASED is digits with randomly selected colour background. Targets represent the colour
train_bg_unbiased_colour_mnist_model = train_all  # when False, automatically loads a trained model
bg_unbiased_colour_mnist_model_to_load     = '../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_bg_unbiased_colour_mnist.weights'
#bg_unbiased_std_colour_mnist_model_to_load = '../exp1_ms_with_random_dataset/results_1g/2024-08-11_11-32-44_SEED104_EPOCHS50_BGN0.1_exp1g_VGG19_bg_unbiased_colour_mnist.weights'

# BIASED is digits with consistent per-class colour background.
train_biased_colour_mnist_model = train_all  # when False, automatically loads a trained model
biased_colour_mnist_model_to_load = '../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_biased_colour_mnist.weights'

# UNBIASED is digits with randoly selected colour background. Targets are digit values
train_unbiased_colour_mnist_model = train_all  # when False, automatically loads a trained model
unbiased_colour_mnist_model_to_load = '2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_unbiased_colour_mnist.weights'

original_train_epochs = 50
bg_noise = 0.1
synthetic_dataset_noise = 0.1

stitch_train_epochs = 50
#stitch_train_loss_tol = 40

device = 'cuda'


In [4]:
# Generate filenames and log the setup details
formatted_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

filename_prefix = f"./{results_root}/{formatted_time}_SEED{seed}_EPOCHS{original_train_epochs}_BGN{bg_noise}_exp2h_VGG19"
save_mix_mnist_model_as = f"{filename_prefix}_mix_mnist.weights"
save_bw_mnist_model_as = f"{filename_prefix}_bw_mnist.weights"
save_bg_only_colour_mnist_model_as = f"{filename_prefix}_bg_only_colour_mnist.weights"
save_bg_unbiased_colour_mnist_model_as = f"{filename_prefix}_bg_unbiased_colour_mnist.weights"
save_biased_colour_mnist_model_as = f"{filename_prefix}_biased_colour_mnist.weights"
save_unbiased_colour_mnist_model_as = f"{filename_prefix}_unbiased_colour_mnist.weights"
save_randinit_model_as = f"{filename_prefix}_randinit.weights"

save_log_as = f"{filename_prefix}_log.txt"

colour_mnist_shape = (3,28,28)
vgg19_input_shape = (3,32,32) # monochrome

logtofile(f"Executed at {formatted_time}")
logtofile(f"logging to {save_log_as}")
logtofile(f"{seed=}")
logtofile(f"{bg_noise=}")
logtofile(f"{synthetic_dataset_noise=}")

logtofile(f"{gen_randinit_model=}")
if gen_randinit_model:
    logtofile(f"{save_randinit_model_as=}")
else:
    logtofile(f"{randinit_model_to_load=}")

logtofile(f"{train_mix_mnist_model=}")
if train_mix_mnist_model:
    logtofile(f"{save_mix_mnist_model_as=}")
    logtofile(f"{original_train_epochs=}")
else:
    logtofile(f"{mix_mnist_model_to_load=}")

logtofile(f"{train_bw_mnist_model=}")
if train_bw_mnist_model:
    logtofile(f"{save_bw_mnist_model_as=}")
    logtofile(f"{original_train_epochs=}")
else:
    logtofile(f"{bw_mnist_model_to_load=}")

logtofile(f"{train_bg_only_colour_mnist_model=}")
if train_bg_only_colour_mnist_model:
    logtofile(f"{save_bg_only_colour_mnist_model_as=}")
    logtofile(f"{original_train_epochs=}")
else:
    logtofile(f"{bg_only_colour_mnist_model_to_load=}")

logtofile(f"{train_bg_unbiased_colour_mnist_model=}")
if train_bg_unbiased_colour_mnist_model:
    logtofile(f"{save_bg_unbiased_colour_mnist_model_as=}")
    logtofile(f"{original_train_epochs=}")
else:
    logtofile(f"{bg_unbiased_colour_mnist_model_to_load=}")

logtofile(f"{train_biased_colour_mnist_model=}")
if train_biased_colour_mnist_model:
    logtofile(f"{save_biased_colour_mnist_model_as=}")
    logtofile(f"{original_train_epochs=}")
else:
    logtofile(f"{biased_colour_mnist_model_to_load=}")

logtofile(f"{train_unbiased_colour_mnist_model=}")
if train_unbiased_colour_mnist_model:
    logtofile(f"{save_unbiased_colour_mnist_model_as=}")
    logtofile(f"{original_train_epochs=}")
else:
    logtofile(f"{unbiased_colour_mnist_model_to_load=}")

logtofile(f"{stitch_train_epochs=}")
logtofile(f"================================================")

Executed at 2025-05-20_09-12-45
logging to ./results_2j_b/2025-05-20_09-12-45_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_log.txt
seed=107
bg_noise=0.1
synthetic_dataset_noise=0.1
gen_randinit_model=False
randinit_model_to_load='2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit.weights'
train_mix_mnist_model=False
mix_mnist_model_to_load='../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_mix_mnist.weights'
train_bw_mnist_model=False
bw_mnist_model_to_load='../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_bw_mnist.weights'
train_bg_only_colour_mnist_model=False
bg_only_colour_mnist_model_to_load='../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_bg_only_colour_mnist.weights'
train_bg_unbiased_colour_mnist_model=False
bg_unbiased_colour_mnist_model_to_load='../exp1_ms_with_random_dataset/results_1g/2025-01-07_10-01-18_SEED107_EPOCHS50_BGN

mnist and cifar-10 both use 10-classes, with 60_000 train samples and 10_000 test samples.

In [5]:
# Set up dataloaders
transform_resize=transforms.Resize(vgg19_input_shape[2])
transform_bw = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transform_resize,
    transforms.ToTensor(),  # convert to tensor. We always do this one
    transforms.Normalize((0.1307,) * 3, (0.3081,) * 3)
])


mnist_train = MNIST("./MNIST", train=True, download=True, transform=transform_bw)
mnist_test = MNIST("./MNIST", train=False, download=True, transform=transform_bw)

bw_train_dataloader = DataLoader(mnist_train, batch_size=64, shuffle=True, drop_last=True)
bw_test_dataloader  = DataLoader(mnist_test,  batch_size=64, shuffle=True, drop_last=False)

# mix dataloader
mix_train_dataloader = colour_mnist.get_mixed_mnist_dataloader(root="./MNIST", batch_size=64, train=True, bg_noise_level=bg_noise, standard_getitem=True, dl_transform=transform_resize)
mix_test_dataloader = colour_mnist.get_mixed_mnist_dataloader(root="./MNIST", batch_size=64,  train=False, bg_noise_level=bg_noise, standard_getitem=True, dl_transform=transform_resize)

# bg_only means no digits - we will use colour as label
bg_only_train_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=1.0, train=True, bg_noise_level=bg_noise, bg_only=True, standard_getitem=True, dl_transform=transform_resize)
bg_only_test_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=1.0, train=False, bg_noise_level=bg_noise, bg_only=True, standard_getitem=True, dl_transform=transform_resize)

# unbiased means each digit has correct label and random colour - but bg means we will use colour as label (i.e. the bias_target will be the target)
bg_unbiased_train_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=0.1, train=True, bg_noise_level=bg_noise, bias_targets_as_targets=True, dl_transform=transform_resize)
bg_unbiased_test_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=0.1, train=False, bg_noise_level=bg_noise, bias_targets_as_targets=True, dl_transform=transform_resize)

# biased means each digit has correct label and consistent colour - Expect network to learn the colours only
biased_train_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=1.0, train=True, bg_noise_level=bg_noise, standard_getitem=True, dl_transform=transform_resize)
biased_test_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=1.0, train=False, bg_noise_level=bg_noise, standard_getitem=True, dl_transform=transform_resize)

# unbiased means each digit has correct label and random colour - Expect network to disregard colours?
unbiased_train_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=0.1, train=True, bg_noise_level=bg_noise, standard_getitem=True, dl_transform=transform_resize)
unbiased_test_dataloader = colour_mnist.get_biased_mnist_dataloader(root="./MNIST", batch_size=64, data_label_correlation=0.1, train=False, bg_noise_level=bg_noise, standard_getitem=True, dl_transform=transform_resize)

## Set up VGG19 models and train it on versions of MNIST

In [6]:

process_structure = dict()


process_structure["randinit"]    = dict()
process_structure["unbias"]    = dict()

# "randinit"
process_structure["randinit"]["model"] = torchvision.models.vgg19(weights=None, num_classes=10).to(device) # Untrained model
process_structure["randinit"]["train"] = gen_randinit_model
process_structure["randinit"]["train_loader"] = biased_train_dataloader
process_structure["randinit"]["test_loader"] = biased_test_dataloader  # by default, test it against the unbias data set
process_structure["randinit"]["saveas"] = save_randinit_model_as
process_structure["randinit"]["loadfrom"] =  randinit_model_to_load

# "unbiased_colour_mnist"
process_structure["unbias"]["model"] = torchvision.models.vgg19(weights=None, num_classes=10).to(device) # Untrained model
process_structure["unbias"]["train"] = train_unbiased_colour_mnist_model
process_structure["unbias"]["train_loader"] = biased_train_dataloader
process_structure["unbias"]["test_loader"] = biased_test_dataloader
process_structure["unbias"]["saveas"] = save_unbiased_colour_mnist_model_as
process_structure["unbias"]["loadfrom"] =  unbiased_colour_mnist_model_to_load


for key, val in process_structure.items():
    print(f"Processing for {key=}")
    if key == "randinit":
        if gen_randinit_model:  # create new model but don't train it
            logtofile(f"model has already been initialised: save it as {val['saveas']}")
            torch.save(val["model"].state_dict(), val["saveas"])
        else:
            logtofile(f"{val['loadfrom']=}")
            val["model"].load_state_dict(torch.load(val["loadfrom"], map_location=torch.device(device)))
    else:
        if val["train"]:
            train_model(model=val["model"], train_loader=val["train_loader"],
                        epochs=original_train_epochs, saveas=val["saveas"],
                        description=key, device=device, logtofile=logtofile,
                        milestones=[100,150],
                        lr=0.01, #0.1 didn't work for some datasets
                       )
        else:
            logtofile(f"{val['loadfrom']=}")
            val["model"].load_state_dict(torch.load(val["loadfrom"], map_location=torch.device(device)))
    val["model"].eval()


Processing for key='randinit'
val['loadfrom']='2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit.weights'
Processing for key='unbias'
val['loadfrom']='2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_unbiased_colour_mnist.weights'


## Measure the Accuracy, Record the Confusion Matrix


In [7]:


original_accuracy = dict()
TDL = biased_test_dataloader
for key, val in process_structure.items():
    logtofile(f"Accuracy Calculation for VGG19 with model {key=} against dataset biased_test_dataloader")
    model = val["model"]
    model.eval() # ALWAYS DO THIS BEFORE YOU EVALUATE MODELS

    # Compute the model accuracy on the test set
    correct = 0
    total = 0

    # assuming 10 classes
    # rows represent actual class, columns are predicted
    confusion_matrix = torch.zeros(10,10, dtype=torch.int)

    for data in TDL:
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)
        predictions = torch.argmax(model(inputs),1)

        matches = predictions == labels
        correct += matches.sum().item()
        total += len(labels)
        for idx, l in enumerate(labels):
            confusion_matrix[l, predictions[idx]] = 1 + confusion_matrix[l, predictions[idx]]

    logtofile("Test the Trained VGG19")
    acc = ((100.0 * correct) / total)
    logtofile('Test Accuracy: %2.2f %%' % acc)
    original_accuracy[key] = acc
    logtofile('Confusion Matrix')
    logtofile(confusion_matrix)
    logtofile(confusion_matrix.sum())

    if not measure_rank:

        dl = TDL

        if val["train"]:
            filename = val["saveas"]
        else:
            filename = val["loadfrom"]
        assert os.path.exists(filename)

        #model = torchvision.models.vgg19(weights=None, num_classes=10)  # Untrained model

        #state = torch.load(filename, map_location=torch.device("cpu"))
        #model.load_state_dict(state, assign=True)
        #model = model.to(device)

        #model = RcvVGG19(model, -1, vgg19_input_shape, device).to(device)
        out_filename = filename.split('/')[-1].replace('.weights', '-test.csv')

        outpath = f"./{results_root}_rank/{key}-bias-{seed}_{out_filename}"  # denote output name as <model_training_type>-dataset-<name>

        if os.path.exists(f"{outpath}"):
            logtofile(f"Already evaluated for {outpath}")
            continue
        #logtofile(f"Measure Rank for {key=}")
        print(f"output to {outpath}")

        params = {}
        params["model"] = key
        params["dataset"] = "bias"
        params["seed"] = seed
        if val["train"]: # as only one network used, record its filename as both send and receive files
            params["send_file"] = val["saveas"]
            params["rcv_file"] = val["saveas"]
        else:
            params["send_file"] = val["loadfrom"]
            params["rcv_file"] = val["loadfrom"]
        params["val_acc"] = acc / 100
        params["name"] = "only"

        results = []
        results.append(params)
        df = pd.DataFrame.from_records(results)
        df.to_csv(f"{outpath}")

        del  params, df
        gc.collect()

logtofile(f"{original_accuracy=}")

Accuracy Calculation for VGG19 with model key='randinit' against dataset biased_test_dataloader
Test the Trained VGG19
Test Accuracy: 10.32 %
Confusion Matrix
tensor([[   0,    0,  980,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0, 1135,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0, 1032,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0, 1010,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,  982,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,  892,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,  958,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0, 1028,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,  974,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0, 1009,    0,    0,    0,    0,    0,    0,    0]],
       dtype=torch.int32)
tensor(10000)
Already evaluated for ./results_2j_b_rank/randinit-bias-107_2025-03-26_18-57-22_SEED107_EPOCHS50_BGN

## Measure Rank with bias dataloader (test) before cutting and stitching

In [8]:
# For the Whole Model - but we will pass it through the RcvResNet18 function to get matching feature names
if measure_rank:
    for key, val in process_structure.items():

        dl = biased_test_dataloader

        if val["train"]:
            filename = val["saveas"]
        else:
            filename = val["loadfrom"]
        assert os.path.exists(filename)

        model = torchvision.models.vgg19(weights=None, num_classes=10)  # Untrained model

        state = torch.load(filename, map_location=torch.device("cpu"))
        model.load_state_dict(state, assign=True)
        model = model.to(device)

        model = RcvVGG19(model, -1, vgg19_input_shape, device).to(device)
        out_filename = filename.split('/')[-1].replace('.weights', '-test.csv')

        outpath = f"./{results_root}_rank/{key}-bias-{seed}_{out_filename}"  # denote output name as <model_training_type>-dataset-<name>

        if os.path.exists(f"{outpath}"):
            logtofile(f"Already evaluated for {outpath}")
            continue
        logtofile(f"Measure Rank for {key=}")
        print(f"output to {outpath}")

        params = {}
        params["model"] = key
        params["dataset"] = "bias"
        params["seed"] = seed
        if val["train"]: # as only one network used, record its filename as both send and receive files
            params["send_file"] = val["saveas"]
            params["rcv_file"] = val["saveas"]
        else:
            params["send_file"] = val["loadfrom"]
            params["rcv_file"] = val["loadfrom"]
        with torch.no_grad():
            layers, features, handles = install_hooks(model)

            metrics = evaluate_model(model, dl, 'acc', verbose=2, device=device)
            params.update(metrics)
            classes = None
            df = perform_analysis(features, classes, layers, params, n=8000)
            df.to_csv(f"{outpath}")

        for h in handles:
            h.remove()
        del model, layers, features, metrics, params, df, handles
        gc.collect()

# Stitch at a given layer


## Train the stitch layer

In [9]:
if False:
    layer_to_cut_after = 1
    device = "cpu"
    model = torchvision.models.vgg19(weights=None, num_classes=10).to(device)  # Untrained model
    #model.load_state_dict(torch.load(filename, map_location=torch.device(device)))  # uses either the load/save name depending whether it'
    cut_layer_output_size = get_layer_output_shape(model, layer_to_cut_after, vgg19_input_shape, device, type="VGG19")
    model_cut = RcvVGG19(model, layer_to_cut_after, vgg19_input_shape, device).to(device)
    print(model_cut)

In [10]:
stitching_accuracies = dict()
stitching_penalties = dict()

for key, val in process_structure.items():
    stitching_accuracies[key] = dict()
    stitching_penalties[key] = dict()
    for layer_to_cut_after in [1,8,22,29,35]:  #  [1,3,6,8,11,13,15,17,20,22,24,26,29,31,33,35]
        ###################### Don't bother to stitch and train if we've already analysed it

        if val["train"]:
            filename = val["saveas"]
        else:
            filename = val["loadfrom"]
        rank_filename = filename.split('/')[-1].replace('.weights', '-test.csv')
        # denote output name as <model_training_type>-dataset-<name>
        # where <model_training_type> is [sender_model or X][layer_to_cut_after][Receiver_model]
        model_training_type = f"{key}{layer_to_cut_after}unbias"
        dataset_type = "bias"

        outpath = f"./{results_root}_rank/{model_training_type}-{dataset_type}-{seed}_{rank_filename}"
        #PENGUIN

        if os.path.exists(f"{outpath}"):
            logtofile(f"Already evaluated for {outpath}")
            continue
        ####################################################################################
        logtofile(f"Evaluate ranks and output to {outpath}")
        logtofile(f"stitch from model {key}")

#        model = torchvision.models.vgg19(weights=None, num_classes=10).to(device)  # Untrained model
#        model.load_state_dict(torch.load(filename, map_location=torch.device(device)))  # uses either the load/save name depending whether it'
#        cut_layer_output_size = get_layer_output_shape(model, layer_to_cut_after, vgg19_input_shape, device, type="VGG19")
#        model_cut = RcvVGG19(model, layer_to_cut_after, vgg19_input_shape, device).to(device)

        # Always use the standard model as receiver
        rcv_model = torchvision.models.vgg19(weights=None, num_classes=10).to(device)  # Untrained model
        rcv_model.load_state_dict(torch.load(unbiased_colour_mnist_model_to_load, map_location=torch.device(device)))
        rcv_model.eval()

        # train a stitch on the unbiased_colour dataset to compare receiver network performance with stitched
        model_stitched = StitchedVGG19(send_model=val["model"],
                                          after_layer_index=layer_to_cut_after,
                                          rcv_model=rcv_model,
                                          input_image_shape=vgg19_input_shape, device=device  ).to(device)

        #############################################################
        # store the initial stitch state
        initial_stitch_weight = model_stitched.stitch.s_conv1.weight.clone()
        initial_stitch_bias   = model_stitched.stitch.s_conv1.bias.clone()
        stitch_initial_weight_outpath    = f"./{results_root}/STITCH_initial_weight_{model_training_type}-{dataset_type}-{seed}_{filename.split('/')[-1]}"
        stitch_initial_bias_outpath      = f"./{results_root}/STITCH_initial_bias_{model_training_type}-{dataset_type}-{seed}_{filename.split('/')[-1]}"
        if save_stitch_delta:
            torch.save(initial_stitch_weight, stitch_initial_weight_outpath)
            torch.save(initial_stitch_bias, stitch_initial_bias_outpath)
        ############################################################

        # define the loss function and the optimiser
        loss_function = nn.CrossEntropyLoss()
        optimiser = optim.SGD(model_stitched.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.01)

        # Put top model into train mode so that bn and dropout perform in training mode
        model_stitched.train()
        # Freeze the top model
        model_stitched.requires_grad_(False)
        # Un-Freeze the stitch layer
        for name, param in model_stitched.stitch.named_parameters():
            param.requires_grad_(True)
        logtofile(f"Train the stitch after layer {layer_to_cut_after}")
        # the epoch loop: note that we're training the whole network

        for epoch in range(stitch_train_epochs):
            running_loss = 0.0
            for data in process_structure[key]["train_loader"]:
                # data is (representations, labels) tuple
                # get the inputs and put them on the GPU
                inputs, labels = data

                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimiser.zero_grad()

                # forward + loss + backward + optimise (update weights)
                outputs = model_stitched(inputs)
                loss = loss_function(outputs, labels)
                loss.backward()
                optimiser.step()

                # keep track of the loss this epoch
                running_loss += loss.item()
            logtofile("Epoch %d, loss %4.2f" % (epoch, running_loss))
        logtofile('**** Finished Training ****')

        model_stitched.eval() # ALWAYS DO THIS BEFORE YOU EVALUATE MODELS

        ############################################################
        # store the trained stitch
        trained_stitch_weight = model_stitched.stitch.s_conv1.weight.clone()
        trained_stitch_bias   = model_stitched.stitch.s_conv1.bias.clone()
        stitch_trained_weight_outpath    = f"./{results_root}/STITCH_trained_weight_{model_training_type}-{dataset_type}-{seed}_{filename.split('/')[-1]}"
        stitch_trained_bias_outpath      = f"./{results_root}/STITCH_trained_bias_{model_training_type}-{dataset_type}-{seed}_{filename.split('/')[-1]}"

        if save_stitch_delta:
            torch.save(trained_stitch_weight, stitch_trained_weight_outpath)
            torch.save(trained_stitch_bias, stitch_trained_bias_outpath)

        print(f"Number of weight / bias in stitch layer is {len(initial_stitch_weight)}")
        stitch_weight_diff = trained_stitch_weight - initial_stitch_weight
        stitch_weight_delta = torch.linalg.norm(stitch_weight_diff).item()
        logtofile(f"Change in stitch weights: {stitch_weight_delta}")
        maxabsweight =  torch.max(stitch_weight_diff.abs()).item()
        logtofile(f"Largest abs weight change: {maxabsweight}")
        stitch_weight_number = torch.sum(torch.where(stitch_weight_diff.abs() > 0.1*maxabsweight, True, False)).item()
        logtofile(f"Number of weights changing > 0.1 of that: {stitch_weight_number}")

        stitch_bias_diff = trained_stitch_bias - initial_stitch_bias
        stitch_bias_delta = torch.linalg.norm(stitch_bias_diff).item()
        logtofile(f"Change in stitch bias: {stitch_bias_delta}")
        maxabsbias =  torch.max(stitch_bias_diff.abs()).item()
        logtofile(f"Largest abs bias change: {maxabsbias}")
        stitch_bias_number = torch.sum(torch.where(stitch_bias_diff.abs() > 0.1*maxabsbias, True, False)).item()
        logtofile(f"Number of bias changing > 0.1 of that: {stitch_bias_number}")

        #new_tensor = torch.load("foo_1_state.pt")
        ##############################################################

        logtofile("Test the trained stitch")
        # Compute the model accuracy on the test set
        correct = 0
        total = 0

        # assuming 10 classes
        # rows represent actual class, columns are predicted
        confusion_matrix = torch.zeros(10,10, dtype=torch.int)
        TDL = process_structure[key]["test_loader"]
        for data in TDL:
            inputs, labels = data
            #print(inputs)
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = torch.argmax(model_stitched(inputs),1)
            #print(model_cut(inputs))
            matches = predictions == labels.to(device)
            correct += matches.sum().item()
            total += len(labels)

            for idx, l in enumerate(labels):
                confusion_matrix[l, predictions[idx]] = 1 + confusion_matrix[l, predictions[idx]]
        acc = ((100.0 * correct) / total)
        logtofile('Test Accuracy: %2.2f %%' % acc)
        logtofile('Confusion Matrix')
        logtofile(confusion_matrix)
        logtofile("===================================================================")
        stitching_accuracies[key][layer_to_cut_after] = acc
        stitching_penalties[key][layer_to_cut_after] = original_accuracy[key] - acc

        if measure_rank:
            dl = TDL
            params = {}
            params["model"] = model_training_type
            params["dataset"] = dataset_type
            params["seed"] = seed
            if val["train"]: # as only one network used, record its filename as both send and receive files
                params["send_file"] = val["saveas"]
                params["rcv_file"] = val["saveas"]
            else:
                params["send_file"] = val["loadfrom"]
                params["rcv_file"] = val["loadfrom"]
            params["stitch_weight_delta"] = stitch_weight_delta
            params["stitch_bias_delta"] = stitch_bias_delta
            params["stitch_weight_number"] = stitch_weight_number
            params["stitch_bias_number"] = stitch_bias_number
            params["my_acc"] = acc

            with torch.no_grad():
                layers, features, handles = install_hooks(model_stitched)

                metrics = evaluate_model(model_stitched, dl, 'acc', verbose=2, device=device)
                params.update(metrics)
                classes = None
                df = perform_analysis(features, classes, layers, params, n=0)
                df.to_csv(f"{outpath}")

            for h in handles:
                h.remove()
            del model_stitched, layers, features, metrics, params, df, handles
            gc.collect()
        else:
            dl = TDL

            print(f"output to {outpath}")

            params = {}
            params["model"] = model_training_type
            params["dataset"] = dataset_type
            params["seed"] = seed
            if val["train"]: # as only one network used, record its filename as both send and receive files
                params["send_file"] = val["saveas"]
                params["rcv_file"] = val["saveas"]
            else:
                params["send_file"] = val["loadfrom"]
                params["rcv_file"] = val["loadfrom"]
            params["val_acc"] = acc / 100
            params["name"] = "only"

            results = []
            results.append(params)
            df = pd.DataFrame.from_records(results)
            df.to_csv(f"{outpath}")

            del  params, df
            gc.collect()


Already evaluated for ./results_2j_b_rank/randinit1unbias-bias-107_2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit-test.csv
Already evaluated for ./results_2j_b_rank/randinit8unbias-bias-107_2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit-test.csv
Already evaluated for ./results_2j_b_rank/randinit22unbias-bias-107_2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit-test.csv
Already evaluated for ./results_2j_b_rank/randinit29unbias-bias-107_2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit-test.csv
Already evaluated for ./results_2j_b_rank/randinit35unbias-bias-107_2025-03-26_18-57-22_SEED107_EPOCHS50_BGN0.1_exp2h_VGG19_randinit-test.csv
Already evaluated for ./results_2j_b_rank/unbias1unbias-bias-107_2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_unbiased_colour_mnist-test.csv
Evaluate ranks and output to ./results_2j_b_rank/unbias8unbias-bias-107_2025-01-07_10-01-18_SEED107_EPOCHS50_BGN0.1_exp1g_VGG19_unbiased_col

In [11]:

stitch_weight_diff = trained_stitch_weight - initial_stitch_weight
stitch_weight_delta = torch.linalg.norm(stitch_weight_diff).item()
print(f"Change in stitch weights: {stitch_weight_delta}")
maxabsweight =  torch.max(stitch_weight_diff.abs()).item()
print(f"Largest abs weight change: {maxabsweight}")
stitch_weight_number = torch.sum(torch.where(stitch_weight_diff.abs() > 0.1*maxabsweight, True, False)).item()
print(f"Number of weights changing > 0.1 of that: {stitch_weight_number}")

print(stitch_weight_delta)
print(stitch_weight_number)

Change in stitch weights: 13.040953636169434
Largest abs weight change: 0.06406603753566742
Number of weights changing > 0.1 of that: 223981
13.040953636169434
223981


In [12]:
logtofile(f"{stitching_accuracies=}")
logtofile(f"{stitching_penalties=}")

stitching_accuracies={'randinit': {}, 'unbias': {8: 99.18, 22: 98.91, 29: 98.77, 35: 98.66}}
stitching_penalties={'randinit': {}, 'unbias': {8: -0.7700000000000102, 22: -0.5, 29: -0.35999999999999943, 35: -0.25}}


In [13]:
for s_key in stitching_accuracies:
    logtofile(f"{s_key}-unbias")
    logtofile(f"{original_accuracy[s_key]=}")
    logtofile("Stitch Accuracy")
    for layer in stitching_accuracies[s_key]:
        logtofile(f"L{layer}: {stitching_accuracies[s_key][layer]}")
    logtofile("--------------------------")

randinit-unbias
original_accuracy[s_key]=10.32
Stitch Accuracy
--------------------------
unbias-unbias
original_accuracy[s_key]=98.41
Stitch Accuracy
L8: 99.18
L22: 98.91
L29: 98.77
L35: 98.66
--------------------------
